In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [20]:
weekend_Apr_list = [1, 7, 8, 14, 15, 21, 22, 28, 29]
#weekend_Apr_list = [1, 8, 15, 22, 29]

In [21]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in weekend_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['AM_flow_minute'][minute_idx])

In [22]:
len(link_day_minute_Apr_list)

25920

In [23]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 1080)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [24]:
x[:,:2]

matrix([[ 3701.33160294,  3109.28071979],
        [ 2904.32180234,  3891.24285265],
        [ 4902.3279198 ,  4351.79174557],
        [ 5006.86458593,  5681.23998511],
        [ 3734.12227753,  4771.13133877],
        [ 4004.18106334,  4381.57494753],
        [ 3853.4360727 ,  4045.83913133],
        [ 2498.0265144 ,  3328.36270088],
        [ 3836.1406555 ,  3792.44666475],
        [ 3762.25837623,  4004.66056363],
        [ 4523.70698763,  4049.97312387],
        [ 4510.51862058,  4601.05047147],
        [ 1247.9195019 ,  2341.99696391],
        [ 4230.807537  ,  1857.11335249],
        [ 1857.37059317,  1734.92763357],
        [ 3642.18701416,  1592.77466883],
        [ 3609.61035658,  3515.44671611],
        [ 4011.38189023,  3294.9997255 ],
        [ 4516.88182814,  2566.27453384],
        [ 4137.07408166,  2028.60052489],
        [ 1633.69659569,  2016.80919821],
        [ 1182.58034151,  2409.33552363],
        [ 4140.77965683,  4355.76615039],
        [ 4054.62613695,  3655.156

In [25]:
np.size(x, 0), np.size(x, 1)

(24, 1080)

In [26]:
np.size(A,0), np.size(A,1)

(24, 140)

In [27]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.15027341e+06  0.00000000e+00  7.66e+03 1.56e+02  9.46e+05     0s
   1   1.25939627e+06 -2.04314645e+06  1.05e+03 1.27e+02  1.86e+05     0s
   2   6.99522000e+05 -1.08941179e+06  9.04e+01 1.16e+01  3.98e+04     0s
   3   6.33524943e+04 -2.54690698e+05  9.04e-05 5.68e-14  5.6

In [28]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekend.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [29]:
lam_list

[8.964554885740308e-10,
 3773.939271226639,
 9.844185722588225e-10,
 1156.1164860778542,
 1154.6775330503447,
 43.277160520879306,
 2.9706693231923104e-07,
 0.00641723019852234,
 1.2179815688282382e-07,
 3377.3487035427047,
 1.2188304730282917e-07,
 1.218600577745175e-07,
 1.217757250208624e-07,
 1.2249901470293265e-07,
 3507.289904500509,
 783.668753175715,
 254.79422545679026,
 1537.134821662515,
 1679.3220028135415,
 3.653298845063366e-05,
 49.40072839599571,
 1626.51762154985,
 1114.2623431829065,
 18.294534632173793,
 2814.089022174501,
 17.184350569877722,
 21.129421841498356,
 2522.233711416672,
 1.8215889006608705,
 1.7726589635750187,
 1202.8206515013985,
 86.02078770462424,
 1879.4490183318906,
 1520.1153005264455,
 723.3933086265813,
 1766.4097926005993,
 611.9409545312542,
 9.783287868070205e-05,
 1599.043797511056,
 29.872638614930793,
 1255.0532878024849,
 557.5062284275581,
 8.155593988615868e-06,
 8.155554871797129e-06,
 2606.9543596859717,
 263.21114907387573,
 841.248